# Feature Engineering Notebook

In this section, we'll conduct essential data transformations guided by the insights gleaned from our previous step, Exploratory Data Analysis (EDA). Additionally, we'll derive new features from existing ones to enhance our understanding of the dataset. The outcome of this phase will be a refined dataset tailored for our machine learning anomaly detection modeling.

#### Import libraries section

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import RobustScaler

### 1. Loading data from staged

In [2]:
consolidated_df = pd.read_csv("../Data/Staged/staged_consolidated_data.csv")
filtered_df = pd.read_csv("../Data/Staged/staged_filtered_data.csv")

In [3]:
consolidated_df.head()

,end_date,year,ticker,form,cluster_6,cluster_10,cluster_15,cluster_14,cluster_4,cluster_2,...,cluster_3,cluster_9,cluster_5,cluster_1,cluster_13,cluster_0,cluster_11,cluster_7,cluster_16,anomaly
0,2008-01-01,2008,CAKE,10-K,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,-0.475738,0.000000,0.000000,0.0,-2.981119e-01,0.000000,0.000000,0.0,0.000000,0
1,2008-01-31,2008,MDLZ,10-K,0.000000,-0.038235,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000,0.000000,0.0,0.000000,0
2,2008-02-02,2008,TGT,10-K,1.373144,6.981317,0.0,1.181053,0.0,4.596254,...,2.330604,0.593854,-0.341399,0.0,-1.040000e-08,-0.088416,17.912556,0.0,2.593812,1
3,2008-02-02,2008,TGT,10-K/A,1.373144,6.981317,0.0,1.181053,0.0,4.596254,...,2.330604,0.593854,-0.341399,0.0,-1.040000e-08,-0.088416,17.912556,0.0,2.593812,1
4,2008-02-02,2008,TGT,10-Q,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,-0.160446,0.000000,0.000000,0.0,0.000000e+00,-0.088416,0.000000,0.0,0.000000,1


In [4]:
numerical_cols_consolidated_df = consolidated_df.columns.tolist()[4:-1]

In [5]:
filtered_df.head()

,end_date,year,ticker,form,Assets,EarningsPerShareBasic,EarningsPerShareDiluted,LiabilitiesAndStockholdersEquity,NetIncomeLoss,RetainedEarningsAccumulatedDeficit,StockholdersEquity,anomaly
0,2008-01-01,2008,CAKE,10-K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2008-01-31,2008,MDLZ,10-K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2008-02-02,2008,TGT,10-K,NaN,0.853175,0.881148,NaN,NaN,NaN,NaN,1
3,2008-02-02,2008,TGT,10-K/A,NaN,0.853175,0.881148,NaN,NaN,NaN,NaN,1
4,2008-02-02,2008,TGT,10-Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [6]:
numerical_cols_filtered_df = filtered_df.columns.tolist()[4:-1]

### 2. Handling outliers

Capping or Clipping: Limit the extreme values within a certain percentile range to reduce their impact.

Both datasets need this step

In [7]:
# (1) Consolidated dataset

for col in numerical_cols_consolidated_df:
    lower_bound = consolidated_df[col].quantile(0.05)
    upper_bound = consolidated_df[col].quantile(0.95)
    consolidated_df[col] = np.clip(consolidated_df[col], lower_bound, upper_bound)

In [8]:
# (2) Filtered dataset

for col in numerical_cols_filtered_df:
    lower_bound = filtered_df[col].quantile(0.05)
    upper_bound = filtered_df[col].quantile(0.95)
    filtered_df[col] = np.clip(filtered_df[col], lower_bound, upper_bound)

### 3. Log transformation
This step will be made to reduce the skewness observed in the numerical features.

In [9]:
# (1) Consolidated dataset

# Log Transformation to reduce skewness
consolidated_df[numerical_cols_consolidated_df] = consolidated_df[numerical_cols_consolidated_df].apply(lambda x: np.log1p(x))

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [10]:
# (2) Filtered dataset

# Log Transformation to reduce skewness
filtered_df[numerical_cols_filtered_df] = filtered_df[numerical_cols_filtered_df].apply(lambda x: np.log1p(x))

### 4. Standarize features
Use robust scaling methods that are less sensitive to outliers, such as RobustScaler which scales based on the median and interquartile range.

Only the 2nd dataset needs this step: filtered_df

In [11]:
# (2) Filtered dataset

# Standardize the features
scaler = RobustScaler()
X_scaled = scaler.fit_transform(filtered_df[numerical_cols_filtered_df])
filtered_df_scaled = pd.DataFrame(X_scaled, columns=numerical_cols_filtered_df)
filtered_df_scaled["anomaly"]= filtered_df["anomaly"]

In [12]:
filtered_df_scaled

,Assets,EarningsPerShareBasic,EarningsPerShareDiluted,LiabilitiesAndStockholdersEquity,NetIncomeLoss,RetainedEarningsAccumulatedDeficit,StockholdersEquity,anomaly
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,NaN,0.681124,0.704393,NaN,NaN,NaN,NaN,1
3,NaN,0.681124,0.704393,NaN,NaN,NaN,NaN,1
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...
5554,-0.504218,-1.430176,-1.474575,-0.504565,-0.292595,-0.289156,-0.390762,0
5555,0.570299,0.447677,0.470120,0.570277,1.328116,-0.196763,0.578382,0
5556,0.292185,0.743182,0.774970,0.292216,0.546734,1.449525,-0.331026,0
5557,-0.060695,0.249874,0.266877,-0.060715,0.205876,-0.390601,-0.420549,0


### 5. Feature Selection
In this step we will select our features and target for the modelling based on EDA conclusions.

We will only take into consideration the numerical columns.
In the second dataset, we will drop the columns suggested by the correlation between the inputs of the dataset: EarningsPerShareDiluted and LiabilitiesAndStockholdersEquity.

In [13]:
# (1) Consolidated dataset

consolidated_df = consolidated_df.iloc[:,4:]

In [14]:
# (2) Filtered dataset

filtered_df_scaled.drop(["EarningsPerShareDiluted", "LiabilitiesAndStockholdersEquity"], axis=1, inplace=True)

#### 6. Saving the processed dataset

In [17]:
# (1) Consolidated dataset

# Save the dataset for modeling
output_file_path = '../data/processed/processed_consolidated_data.csv'
consolidated_df.to_csv(output_file_path, index=False)
print(f"Processed dataframe saved to {output_file_path}")

Processed dataframe saved to ../data/processed/processed_consolidated_data.csv


In [18]:
# (2) Filtered dataset

# Save the dataset for modeling
output_file_path = '../data/processed/processed_filtered_data.csv'
filtered_df_scaled.to_csv(output_file_path, index=False)
print(f"Processed dataframe saved to {output_file_path}")

Processed dataframe saved to ../data/processed/processed_filtered_data.csv
